In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import os
from pathlib import Path
import sys
from time import time
import numpy as np
import pandas as pd

import sklearn
from collections import OrderedDict
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

SEED = None

# Load data

In [2]:
datadir = Path('../../data/yitan/Data')
ccl_folds_dir = Path('../../data/yitan/CCL_10Fold_Partition')
pdm_folds_dir = Path('../../data/yitan/PDM_10Fold_Partition')

In [3]:
# Un-pickle files
import _pickle as cp

pkl_file = open(datadir/'CCL_PDM_TransferLearningData_rmFactor_0.0_ddNorm_std.pkl', 'rb')

res = cp.load(pkl_file)
genomics = cp.load(pkl_file)
drug = cp.load(pkl_file)

pkl_file.close()

In [4]:
print('res     ', res.shape)
print('genomics', genomics.shape)
print('drug    ', drug.shape)

res      (708662, 5)
genomics (1430, 4582)
drug     (1402, 4392)


# First look at the data

In [29]:
display(res[:2])
display(genomics.iloc[:2, :7])
display(drug.iloc[:2, :7])

,SOURCE,ccl_name,ctrpDrugID,area_under_curve,groupID
0,CCLE,CCL_61,Drug_11,0.7153,0.0000
1,CCLE,CCL_61,Drug_1,0.9579,0.9164


,geneGE_AARS,geneGE_ABCB6,geneGE_ABCC5,geneGE_ABCF1,geneGE_ABCF3,geneGE_ABHD4,geneGE_ABHD6
CCL_1,-0.125161,-0.400237,-0.960208,0.575207,-0.468406,-0.136257,0.083319
CCL_10,-0.217106,0.354776,-1.164841,0.328071,-0.735267,0.232990,-0.174979


,DD_MW|num,DD_AMW|num,DD_Sv|num,DD_Se|num,DD_Sp|num,DD_Si|num,DD_Mv|num
Drug_1,0.123446,0.526234,-0.072180,-0.088861,-0.058460,-0.083100,-0.009539
Drug_10,0.053188,1.966100,-0.333843,-0.379081,-0.359584,-0.398841,1.172374


In [6]:
res.groupby('SOURCE').agg({'ccl_name': 'nunique', 'ctrpDrugID': 'nunique'}).reset_index()

,SOURCE,ccl_name,ctrpDrugID
0,CCLE,474,24
1,CTRP,812,494
2,GDSC,670,238
3,NCI60,59,987
4,PDM,473,18
5,gCSI,357,16


### Where are the genomic features coming from?

In [7]:
gen_ = genomics.reset_index().rename(columns={'index': 'ccl_name'})

In [31]:
display(gen_.iloc[:2, :5])
display(res[:2])

,ccl_name,geneGE_AARS,geneGE_ABCB6,geneGE_ABCC5,geneGE_ABCF1
0,CCL_1,-0.125161,-0.400237,-0.960208,0.575207
1,CCL_10,-0.217106,0.354776,-1.164841,0.328071


,SOURCE,ccl_name,ctrpDrugID,area_under_curve,groupID
0,CCLE,CCL_61,Drug_11,0.7153,0.0000
1,CCLE,CCL_61,Drug_1,0.9579,0.9164


In [20]:
aa = pd.merge(gen_[['ccl_name']], res[['SOURCE', 'ccl_name']], on='ccl_name', how='inner')
# aa = aa.drop_duplicates().reset_index(drop=True)
print(aa.shape)
aa['SOURCE'].value_counts()

(708662, 2)


CTRP     325789
NCI60    227357
GDSC     135568
CCLE      10971
gCSI       5647
PDM        3330
Name: SOURCE, dtype: int64

In [32]:
bb = aa[aa['ccl_name']=='CCL_1']['SOURCE'].value_counts()
bb

NCI60    3356
GDSC      181
Name: SOURCE, dtype: int64

### What features are available?

In [6]:
def cnt_feas(df, sep='_', verbose=True):
    """ Count number of unique features types. """
    dct = {}
    for c in df.columns:
        prfx = c.split(sep)[0]
        if prfx in dct.keys():
            dct[prfx] += 1
        else:
            dct[prfx] = 1
            
    if verbose:
        print(dct)
    return dct

In [7]:
gen_dct = cnt_feas(genomics, sep='_')
drg_dct = cnt_feas(drug, sep='_')

{'geneGE': 1927, 'c2cpMaxGE': 1328, 'c2cpMinGE': 1327}
{'DD': 2344, 'ECFP': 1024, 'PFP': 1024}


In [8]:
# Change var names
gen = genomics
drg = drug

In [9]:
res = res.reset_index()
res = res.rename(columns={'index': 'id'})
res[:2]

,id,SOURCE,ccl_name,ctrpDrugID,area_under_curve,groupID
0,0,CCLE,CCL_61,Drug_11,0.7153,0.0000
1,1,CCLE,CCL_61,Drug_1,0.9579,0.9164


In [10]:
def extract_subset_fea(df, fea_list, fea_sep='_'):
    fea = [c for c in df.columns if (c.split(fea_sep)[0]) in fea_list]
    df = df[fea]
    return df

## Some code test

In [11]:
res_ = res.copy()
gen_ = gen.copy()
drg_ = drg.copy()

In [12]:
# Extract specific features
gen_fea_list = ['geneGE']
drg_fea_list = ['DD']

cnt_feas(gen_, sep='_', verbose=True);
gen_ = extract_subset_fea(df=gen_, fea_list=gen_fea_list, fea_sep='_')
cnt_feas(gen_, sep='_', verbose=True);

cnt_feas(drg_, sep='_', verbose=True);
drg_ = extract_subset_fea(df=drg, fea_list=drg_fea_list, fea_sep='_')
cnt_feas(drg_, sep='_', verbose=True);

{'geneGE': 1927, 'c2cpMaxGE': 1328, 'c2cpMinGE': 1327}
{'geneGE': 1927}
{'DD': 2344, 'ECFP': 1024, 'PFP': 1024}
{'DD': 2344}


In [13]:
# Bring the labels in
gen_ = gen_.reset_index().rename(columns={'index': 'ccl_name'})
drg_ = drg_.reset_index().rename(columns={'index': 'ctrpDrugID'})

In [14]:
# Extract source and fold
src = 'CCLE'
fold = 0
path = ccl_folds_dir/f'{src}/cv_{fold}' # 'TestList.txt'

tr_id = pd.read_csv(path/'TrainList.txt', header=None).squeeze().values
vl_id = pd.read_csv(path/'ValList.txt', header=None).squeeze().values
te_id = pd.read_csv(path/'TestList.txt', header=None).squeeze().values

# Show how much is left for train, val, and test
tr_sz, vl_sz, te_sz = len(tr_id), len(vl_id), len(te_id)
sz = tr_sz + vl_sz + te_sz
print(tr_sz/sz)

# Retain specific source and shuffle
print(res_.shape)
res_ = res_[ res['SOURCE'].isin([src]) ]
# res_ = res_.sample(frac=1.0, random_state=42).reset_index(drop=True)
print(res_.shape)

0.7974683544303798
(708662, 6)
(10971, 6)


In [15]:
# Merge data
len(res_.ccl_name.unique())

mrg = pd.merge(res_, gen_, on='ccl_name', how='inner')
mrg = pd.merge(mrg, drg_, on='ctrpDrugID', how='inner')
print(mrg.shape)
display(mrg[:2])

(10971, 4277)


,id,SOURCE,ccl_name,ctrpDrugID,area_under_curve,groupID,geneGE_AARS,geneGE_ABCB6,geneGE_ABCC5,geneGE_ABCF1,...,DD_VE1_B(m)_naLabeled|int,DD_VE3sign_B(m)_naLabeled|int,DD_VE3sign_B(v)_naLabeled|int,DD_VE3sign_B(e)_naLabeled|int,DD_VE3sign_B(p)_naLabeled|int,DD_VE3sign_B(i)_naLabeled|int,DD_VE3sign_B(s)_naLabeled|int,DD_AMR_naLabeled|int,DD_SAtot_naLabeled|int,DD_BLTF96_naLabeled|int
0,0,CCLE,CCL_61,Drug_11,0.7153,0.0000,0.796453,1.185398,-0.279271,-1.612266,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,24,CCLE,CCL_65,Drug_11,0.8126,0.3626,-0.071298,1.047415,-1.032675,0.382738,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# PyTorch data generator

In [16]:
import torch
from torch import nn, optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [17]:
print(res.shape)
print(gen.shape)
print(drg.shape)

(708662, 6)
(1430, 4582)
(1402, 4392)


In [18]:
# drg_df = drg.copy()
# gen_df = gen.copy()

# drg_fea_dct = {idx: row.values for idx, row in drg_df.iterrows()}
# gen_fea_dct = {idx: row.values for idx, row in gen_df.iterrows()}

# res.values

In [19]:
res_df = res.copy()
drg_df = drg.copy()
gen_df = gen.copy()
tr_ph = 'train'
fold = 0
src = 'CCLE'
gen_fea_list = ['geneGE']
drg_fea_list = ['DD']

res_df = res_df[ res_df['SOURCE'].isin([src]) ]
drg_df = extract_subset_fea(drg_df, drg_fea_list, fea_sep='_')
gen_df = extract_subset_fea(gen_df, gen_fea_list, fea_sep='_')

path = ccl_folds_dir/f'{src}/cv_{fold}'

res_arr = res_df.values
drg_fea_dct = {idx: row.values for idx, row in drg_df.iterrows()}
gen_fea_dct = {idx: row.values for idx, row in gen_df.iterrows()}

In [20]:
class CCLDataset(Dataset):
    # discuss.pytorch.org/t/data-processing-as-a-batch-way/14154
    # github.com/utkuozbulak/pytorch-custom-dataset-examples#incorporating-pandas
    # nbviewer.jupyter.org/github/FraPochetti/KagglePlaygrounds/blob/master/NYC%20Taxi%20Fares%20Prediction.ipynb
    
    def __init__(self,
                 res_df: pd.DataFrame,
                 gen_df: pd.DataFrame,
                 drg_df: pd.DataFrame,
                 ccl_folds_dir: str,
                 src: str,
                 fold: int,
                 tr_ph: str,
                 gen_fea_list: list,
                 drg_fea_list: list,
                 drg_dsc_preproc: str=None,   # TODO
                 cell_rna_preproc: str=None,  # TODO
                 verbose: bool=True):
        """ 
        Args:
            res_df (pd.DataFrame) : 
            gen_df (pd.DataFrame) :
            drg_df (pd.DataFrame) :
            ccl_folds_dir (str) :
            src (str) :
            fold (int) :
            tr_ph (str) :
            gen_fea_list (list) :
            drg_fea_list (list) :
            drg_dsc_preproc (str) : TODO: not implemented
            cell_rna_preproc (str) : TODO: not implemented
            verbose: bool=True
        """
        
        # ============================================
        # Initialize
        # ============================================
        self.ccl_folds_dir = ccl_folds_dir
        self.src = src
        self.fold = fold
        self.tr_ph = tr_ph.lower()
        self.gen_fea_list = gen_fea_list
        self.drg_fea_list = drg_fea_list
        self.drg_dsc_preproc = None
        self.cell_rna_preproc = None

        # ============================================
        # Get the indices
        # ============================================
        if self.tr_ph in ['tr', 'train', 'training']:
            self.ids_fname = 'TrainList.txt'
        elif self.tr_ph in ['vl', 'val', 'validation']:
            self.ids_fname = 'ValList.txt'
        elif self.tr_ph in ['te', 'test', 'testing']:
            self.ids_fname = 'TestList.txt'
        else:
            raise ValueError('Wrong `tr_ph` specified.')
            
        self.ids_path = self.ccl_folds_dir/f'{self.src}/cv_{self.fold}'/self.ids_fname # 'TestList.txt'        
        self.ids_list = pd.read_csv(self.ids_path, header=None).squeeze().values
        
        # ============================================
        # Load dataframes
        # ============================================
        res_df = res_df[ res_df['SOURCE'].isin([src]) ]
        self.res_df = res_df[ res_df['ccl_name'].isin( self.ids_list ) ]
        
        self.drg_df = self.extract_subset_fea(drg_df, self.drg_fea_list)
        self.gen_df = self.extract_subset_fea(gen_df, self.gen_fea_list)        
        
        # ============================================
        # Public attributes
        # ============================================
        self.drugs = self.res_df['ctrpDrugID'].unique().tolist()
        self.cells = self.res_df['ccl_name'].unique().tolist()
        self.num_records = len(self.res_df)
        self.drg_dim = self.drg_df.shape[1]
        self.gen_dim = self.gen_df.shape[1]
        
        # ============================================
        # Convert dfs to arrays and dict for faster access
        # ============================================
        self.res_arr = self.res_df.values
        self.drg_fea_dct = {idx: row.values for idx, row in drg_df.iterrows()}
        self.gen_fea_dct = {idx: row.values for idx, row in gen_df.iterrows()}

        # ============================================
        # Summary
        # ============================================
        if verbose:
            print('=' * 80)
            print(f'Data source: {self.src}')
            print(f'Phase: {tr_ph}')
            print(f'Data points: {self.num_records}')
            print(f'Unique drugs: {len(self.drugs)}')
            print(f'Unique cells: {len(self.cells)}')
            print(f'drg_df.shape: {self.drg_df.shape}')
            print(f'gen_df.shape: {self.gen_df.shape}')
            # print(f'Drug features: {self.get_drg_fea_cnt}')
            # print(f'Genomic features: {self.get_gen_fea_cnt}')

        
    def __len__(self):
        return len(self.res_arr)

    
    def __getitem__(self, index):
        """ 
        Ref: github.com/xduan7/UnoPytorch/blob/master/utils/datasets/drug_resp_dataset.py
        Look for __getitem__ in DrugRespDataset
        
        res indices: [id, SOURCE, ccl_name, ctrpDrugID, area_under_curve, groupID]
        """
        
        res = self.res_arr[index]
        
        idx = res[0]
        SOURCE = res[1]
        ccl_id = res[2]
        drg_id = res[3]
        auc = res[4]
        
        ccl_fea = self.gen_fea_dct[ccl_id]
        drg_fea = self.drg_fea_dct[drg_id]
        
        # Cast values
        ccl_fea = ccl_fea.astype(np.float32)
        drg_fea = drg_fea.astype(np.float32)
                
        return idx, SOURCE, ccl_id, drg_id, auc, ccl_fea, drg_fea
    
    
    def extract_subset_fea(self, df, fea_list, fea_sep='_'):
        fea = [c for c in df.columns if (c.split(fea_sep)[0]) in fea_list]
        df = df[fea]
        return df    
    
    
    def get_gen_fea_cnt(self, sep='_', verbose=True):
        """ Count number of unique feature types. """
        dct = {}
        for c in self.gen_df.columns:
            prfx = c.split(sep)[0]
            if prfx in dct.keys():
                dct[prfx] += 1
            else:
                dct[prfx] = 1

        if verbose:
            print(dct)
        
        return dct
    
    
    def get_drg_fea_cnt(self, sep='_', verbose=True):
        """ Count number of unique feature types. """
        dct = {}
        for c in self.drg_df.columns:
            prfx = c.split(sep)[0]
            if prfx in dct.keys():
                dct[prfx] += 1
            else:
                dct[prfx] = 1

        if verbose:
            print(dct)
        
        return dct

In [21]:
gen_fea_list = ['geneGE']
drg_fea_list = ['DD']
tr_ds = CCLDataset(res_df = res,
                   gen_df = gen,
                   drg_df = drg,
                   ccl_folds_dir = ccl_folds_dir,
                   tr_ph = 'train', fold = 0, src = 'CCLE',
                   gen_fea_list = gen_fea_list,
                   drg_fea_list = drg_fea_list)
# vl_ds = CCLDataset(xdata=xte, ydata=yte)
# te_ds = CCLDataset(xdata=xte, ydata=yte)

Data source: CCLE
Phase: train
Data points: 8755
Unique drugs: 24
Unique cells: 378
drg_df.shape: (1402, 2344)
gen_df.shape: (1430, 1927)


In [22]:
# Define data loaders
batch_size = 1
num_workers = 1
tr_loader_prms = {'batch_size': batch_size, 'shuffle': True, 'num_workers': num_workers}
# te_loader_prms = {'batch_size': 4*batch_size, 'shuffle': False, 'num_workers': num_workers}

tr_loader = DataLoader(tr_ds, **tr_loader_prms)
# te_loader = DataLoader(te_ds, **te_loader_prms)

In [23]:
ret = next(iter(tr_loader))  # (idx, SOURCE, ccl_id, drg_id, auc, ccl_fea, drg_fea)

idx = ret[0]
source = ret[1]
cell_id = ret[2]
drug_id = ret[3]
auc = ret[4]
cell_fea = ret[5]
drug_fea = ret[6]


In [24]:
ret

[tensor([6808]),
 ('CCLE',),
 ('CCL_668',),
 ('Drug_7',),
 tensor([0.7728], dtype=torch.float64),
 tensor([[ 0.3722,  0.9731,  0.0231,  ...,  1.2562, -0.0929,  0.7574]]),
 tensor([[ 0.1505,  2.7617, -0.4077,  ...,  0.0000,  1.0000,  0.0000]])]

In [25]:
print( sum(tr_ds.res_df['ccl_name']==ret[2][0]) )
print( sum(tr_ds.res_df['ctrpDrugID']==ret[3][0]) )
sum( (tr_ds.res_df['ccl_name']==ret[2][0]) & (tr_ds.res_df['ctrpDrugID']==ret[3][0]) )

23
378


1

In [26]:
tr_ds.res_df[ (tr_ds.res_df['ccl_name']==ret[2][0]) & (tr_ds.res_df['ctrpDrugID']==ret[3][0]) ]

,id,SOURCE,ccl_name,ctrpDrugID,area_under_curve,groupID
6808,6808,CCLE,CCL_668,Drug_7,0.7728,0.037


In [27]:
tr_ds.res_df.loc[ret[0]]

,id,SOURCE,ccl_name,ctrpDrugID,area_under_curve,groupID
6808,6808,CCLE,CCL_668,Drug_7,0.7728,0.037


In [28]:
len( set(tr_ds.ids_list).intersection(set(tr_id)) )

378

In [18]:
l=['lin', 'a']

any(['lin'==s for s in l])

True

In [21]:
l = ['a', 'b']
l = l + 'c'

TypeError: can only concatenate list (not "str") to list

In [22]:
l

['a', 'b']